<a href="https://colab.research.google.com/github/paolakroetzperetti/GrownSpanishBackups/blob/main/Projeto_jogos_fase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
O código consiste em duas classes principais, GameDataLoader e GameAnalyzer, que são usadas para carregar e analisar dados sobre jogos de videogame.
Ele também inclui uma função principal chamada main que carrega os dados de um arquivo CSV, realiza análises sobre esses dados e imprime os resultados.

Exemplo de Uso:
Suponhamos que você tenha dois arquivos: "steam_games.csv" com os dados completos e "amostras.csv" com uma amostra dos dados.

Carregando e Analisando Dados Completos:

    main("/content/steam_games.csv")

Neste exemplo, o código carregará e analisará os dados completos do arquivo "steam_games.csv". Ele calculará o percentual de jogos gratuitos e pagos, identificará o(s) ano(s) com o maior número de jogos e encontrará o jogo mais antigo. Em seguida, imprimirá os resultados das análises.

Carregando e Analisando Dados de Amostra:

    main("/content/amostras.csv")

Neste exemplo, o código carregará e analisará a amostra dos dados do arquivo "amostras.csv". Ele realizará as mesmas análises que no exemplo anterior e imprimirá os resultados.
"""

class GameDataLoader:
    def __init__(self, filename):
        self.filename = filename

    def load_data(self):
        data = []
        with open(self.filename, "r") as file:
            for line in file:
                # Tratar a coluna "Release date" removendo as vírgulas
                line = self._remove_commas_in_release_date(line)
                data.append(line.strip().split(","))  # Split agora considera vírgulas normais
        return data

    def _remove_commas_in_release_date(self, line):
        parts = line.split('"')  # Divide a linha em partes separadas por aspas
        for i, part in enumerate(parts):
            if i % 2 == 1:  # As partes ímpares são aquelas entre as aspas
                parts[i] = part.replace(',', '')  # Remove vírgulas nas partes ímpares
        return '"'.join(parts)  # Junta as partes de volta usando as aspas

In [4]:
from datetime import datetime

class GameAnalyzer:
    def __init__(self, game_data):
        self.game_data = game_data

    def calcular_percentuais(self):
        total_jogos = len(self.game_data)
        jogos_gratuitos = sum(1 for jogo in self.game_data if len(jogo) > 6 and jogo[6] != '0.0')
        jogos_pagos = total_jogos - jogos_gratuitos
        percentual_gratuitos = (jogos_gratuitos / total_jogos) * 100
        percentual_pagos = (jogos_pagos / total_jogos) * 100
        return percentual_gratuitos, percentual_pagos

    def encontrar_anos_mais_jogos(self):
        anos = {}
        for jogo in self.game_data:
            if len(jogo) > 2:
                release_date = jogo[2].split(", ")[-1]
                if release_date in anos:
                    anos[release_date] += 1
                else:
                    anos[release_date] = 1
        if anos:
            maior_numero_jogos = max(anos.values())
            anos_mais_jogos = [ano for ano, num_jogos in anos.items() if num_jogos == maior_numero_jogos]
        else:
            anos_mais_jogos = []
        return anos_mais_jogos

    def encontrar_jogo_mais_antigo(self):
        jogos_validos = [jogo for jogo in self.game_data if len(jogo) > 2]

        if not jogos_validos:
            raise ValueError("Nenhum jogo válido encontrado.")

        jogo_mais_antigo = min(jogos_validos, key=lambda jogo: jogo[2])
        return jogo_mais_antigo

In [23]:
import html

def main(filename):
    try:
        # Carregando os dados do arquivo usando o GameDataLoader
        loader = GameDataLoader(filename)
        game_data = loader.load_data()
    except FileNotFoundError:
        print("Arquivo de dados não encontrado.")
        exit(1)
    except Exception as e:
        print("Erro ao carregar os dados:", e)
        exit(1)

    # Exibição das 2 primeiras linhas do arquivo para verificar se o mesmo está sendo lido corretamente.
    print("Primeiras 2 linhas do arquivo:")
    try:
        for item in game_data[:2]:
            if item:  # Verificando se a linha não está vazia
                print(item)
    except Exception as e:
        print("Erro ao exibir as linhas do arquivo:", e)
        exit(1)

    # Criando uma instância da classe GameAnalyzer
    try:
        analyzer = GameAnalyzer(game_data)
    except Exception as e:
        print("Erro ao criar a instância do GameAnalyzer:", e)
        exit(1)

    # Decodificar caracteres especiais nos resultados
    def print_decoded_results(results):
        for result in results:
            decoded_result = [html.unescape(item) for item in result]
            print(decoded_result)

    # Pergunta 1: Qual o percentual de jogos gratuitos e pagos na plataforma?
    percentual_pagos, percentual_gratuitos = analyzer.calcular_percentuais()
    percentual_pagos = round(percentual_pagos, 2)
    percentual_gratuitos = round(percentual_gratuitos, 2)

    # Pergunta 2: Qual o ano com o maior número de novos jogos? Em caso de empate, retorne uma lista com os anos empatados.
    anos_mais_jogos = analyzer.encontrar_anos_mais_jogos()

    # Pergunta 3: Qual é o jogo mais antigo?
    jogo_mais_antigo = analyzer.encontrar_jogo_mais_antigo()

    # Impressão dos resultados das perguntas:
    print("\nPergunta 1:")
    print("Percentual de jogos gratuitos:", f"{percentual_gratuitos:.2f}%")
    print("Percentual de jogos pagos:", f"{percentual_pagos:.2f}%")
    print()

    print("Pergunta 2:")
    if len(anos_mais_jogos) == 1:
        print("Ano com o maior número de jogos:", anos_mais_jogos[0])
    else:
        print("Ano(s) com o maior número de jogos:", ", ".join(anos_mais_jogos))
    print()

    print("Pergunta 3:")
    if jogo_mais_antigo is not None:
        print("Jogo mais antigo:")
        print(jogo_mais_antigo)
    else:
        print("Não foi possível encontrar o jogo mais antigo.")

# Executar para o arquivo completo
main("/content/steam_games.csv")

Primeiras 2 linhas do arquivo:
['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU', 'Required age', 'Price', 'DLC count', 'About the game', 'Supported languages', 'Full audio languages', 'Reviews', 'Header image', 'Website', 'Support url', 'Support email', 'Windows', 'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score', 'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations', 'Notes', 'Average playtime forever', 'Average playtime two weeks', 'Median playtime forever', 'Median playtime two weeks', 'Developers', 'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies']
['20200', '"Galactic Bowling"', '"Oct 21 2008"', '"0 - 20000"', '0', '0', '19.99', '0', '"Galactic Bowling is an exaggerated and stylized bowling game with an intergalactic twist. Players will engage in fast-paced single and multi-player competition while being submerged in a unique new universe filled with over-the-top humor wild characters unique levels and addictiv

In [24]:
main("/content/amostras.csv")

Primeiras 2 linhas do arquivo:
['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU', 'Required age', 'Price', 'DLC count', 'About the game', 'Supported languages', 'Full audio languages', 'Reviews', 'Header image', 'Website', 'Support url', 'Support email', 'Windows', 'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score', 'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations', 'Notes', 'Average playtime forever', 'Average playtime two weeks', 'Median playtime forever', 'Median playtime two weeks', 'Developers', 'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies']
['589250', '"PowersVR"', '"Oct 10 2020"', '"0 - 20000"', '0', '0', '9.99', '0', '"PowersVR lets you be the caped crusader from comic books. Featuring a cast of colorful heroes you\'ll get to test out their unique superpowers in Metropolis. Compete in challenges unlock all superpowers fight enemies and discover hidden collectibles. With more heroes and challenges yet t